## import package

In [2]:
import numpy as np
import pandas as pd
import os
import gc


import json
import math 
import sys
from datetime import datetime
import time
from typing import Tuple,List

import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import sklearn.metrics
from PIL import Image

from matplotlib.axes import Axes
from matplotlib import animation, rc
import plotly.graph_objs as go
import plotly.tools as ts
from plotly.offline import plot, init_notebook_mode
import plotly.figure_factory as ft
init_notebook_mode(connected=True)
from pyquaternion import Quaternion
import seaborn as sns
from tqdm import tqdm 
import warnings

from lyft_dataset_sdk.utils.map_mask import MapMask
from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.geometry_utils import view_points, box_in_image, BoxVisibility
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
from pathlib import Path

from lyft_dataset_sdk.lyftdataset import LyftDataset,LyftDatasetExplorer
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
import time
from lyft_dataset_sdk.utils.map_mask import MapMask

import struct
from abc import ABC, abstractmethod
from functools import reduce
from typing import Tuple, List, Dict
import copy
from shutil import copyfile

from PIL import Image
import math

## input

In [3]:
new_classname, new_cond = 'car', 'mirror2'

In [4]:
predicted_logfile = 'predictedLabel_Folder.csv'
df_logfile = pd.read_csv(predicted_logfile)

## load test_dataset

In [5]:
test_dataset = LyftDataset(data_path='.', json_path='/home/lingling/Downloads/3d-object-detection-for-autonomous-vehicles/test_data/', verbose=True)

JSON file instance.json missing, using empty list
JSON file sample_annotation.json missing, using empty list
9 category,
17 attribute,
4 visibility,
0 instance,
8 sensor,
168 calibrated_sensor,
219744 ego_pose,
218 log,
218 scene,
27468 sample,
219744 sample_data,
0 sample_annotation,
1 map,
Done loading in 2.3 seconds.
Reverse indexing ...
Done reverse indexing in 0.6 seconds.


## functions

### label_global_from_rect

#def label_global_from_rect(sample_token,args):
    #args = [x,y,z,width,length,height,yaw]
    # get the label from the sample_ID
    sample = test_dataset.get("sample", sample_token)

    sample_lidar_token = sample["data"]["LIDAR_TOP"]
    lidar_data = test_dataset.get("sample_data", sample_lidar_token)

    # get lidar calibration info
    ego_pose = test_dataset.get("ego_pose", lidar_data["ego_pose_token"])
    calibrated_sensor = test_dataset.get("calibrated_sensor", lidar_data["calibrated_sensor_token"])

    # get the transformation matrix from calibration info
    Tr_rect_sensor = np.array([[-1,0,0,0],[0,0,1,0],[0,-1,0,0],[0,0,0,1]])

    global_from_car = transform_matrix(ego_pose['translation'],
                                       Quaternion(ego_pose['rotation']), inverse=False)
    car_from_sensor = transform_matrix(calibrated_sensor['translation'], Quaternion(calibrated_sensor['rotation']),
                                        inverse=False)
    global_from_sensor = np.dot(global_from_car,car_from_sensor)
    
    global_from_rect = np.dot(global_from_sensor,Tr_rect_sensor)
    
    center_x = args[0]
    center_y = args[1]
    center_z = args[2]
    width = args[3]
    length = args[4]
    height = args[5]
    yaw = args[6]
        
        
        
    # transfrom the center point from global to sensor (center_x,center_y,center_z)->(x,y,z)
    tmp = np.array([[center_x],[center_y],[center_z],[1.0]])
    coor = np.dot(global_from_rect,tmp)
    x = coor[0][0]
    y = coor[1][0]
    z = coor[2][0]

    # transform the yaw from sensor to global yaw->yaw_sensor
    # yaw vector += center vector
    x_yaw = center_x - np.sin(yaw)
    z_yaw = center_z + np.cos(yaw)
    y_yaw = center_y
    # transfer yaw vector to sensor
    coor_yaw = np.dot(global_from_rect,np.array([[x_yaw],[y_yaw],[z_yaw],[1.0]]))
    x_yaw_global = coor_yaw[0][0]
    y_yaw_global = coor_yaw[1][0]
    z_yaw_global = coor_yaw[2][0]
    # the angle between yaw_sensor and the y axis
    #yaw_new = np.arccos(np.clip(np.dot((1,0),(y_yaw_global - y,x_yaw_global - x)), -1.0, 1.0))
    yaw_new = math.atan2(-(x_yaw_global - x),y_yaw_global - y)
    
        
    return [x,y,z,width,length,height,yaw_new]

In [6]:
def label_global_from_rect(sample_token,args):
    #args = [x,y,z,width,length,height,yaw]
    # get the label from the sample_ID
    sample = test_dataset.get("sample", sample_token)

    sample_lidar_token = sample["data"]["LIDAR_TOP"]
    lidar_data = test_dataset.get("sample_data", sample_lidar_token)

    # get lidar calibration info
    ego_pose = test_dataset.get("ego_pose", lidar_data["ego_pose_token"])
    calibrated_sensor = test_dataset.get("calibrated_sensor", lidar_data["calibrated_sensor_token"])

    # get the transformation matrix from calibration info
    Tr_rect_sensor = np.array([[-1,0,0,0],[0,0,1,0],[0,-1,0,0],[0,0,0,1]])

    global_from_car = transform_matrix(ego_pose['translation'],
                                       Quaternion(ego_pose['rotation']), inverse=False)
    car_from_sensor = transform_matrix(calibrated_sensor['translation'], Quaternion(calibrated_sensor['rotation']),
                                        inverse=False)
    global_from_sensor = np.dot(global_from_car,car_from_sensor)
    
    global_from_rect = np.dot(global_from_sensor,Tr_rect_sensor)
    
    center_x = args[0]
    center_y = args[1]
    center_z = args[2]
    width = args[3]
    length = args[4]
    height = args[5]
    yaw = -args[6]
        
        
        
    # transfrom the center point from global to sensor (center_x,center_y,center_z)->(x,y,z)
    tmp = np.array([[center_x],[center_y],[center_z],[1.0]])
    coor = np.dot(global_from_rect,tmp)
    x = coor[0][0]
    y = coor[1][0]
    z = coor[2][0]

    # transform the yaw from sensor to global yaw->yaw_sensor
    # yaw vector += center vector
    x_yaw = center_x - np.sin(yaw)
    z_yaw = center_z + np.cos(yaw)
    y_yaw = center_y
    # transfer yaw vector to sensor
    coor_yaw = np.dot(global_from_rect,np.array([[x_yaw],[y_yaw],[z_yaw],[1.0]]))
    x_yaw_global = coor_yaw[0][0]
    y_yaw_global = coor_yaw[1][0]
    z_yaw_global = coor_yaw[2][0]
    # the angle between yaw_sensor and the y axis
    #yaw_new = np.arccos(np.clip(np.dot((1,0),(y_yaw_global - y,x_yaw_global - x)), -1.0, 1.0))
    yaw_new = math.atan2(-(x_yaw_global - x),y_yaw_global - y)
    
        
    return [x,y,z,width,length,height,yaw_new]


### read_label_predict

In [7]:
def read_label_predict(label_path):
    label=pd.read_csv(label_path,
               sep='\s', 
               names=["confidence","center_x", "center_y", "center_z","width","length","height", "yaw","class_name"])

    numerical_cols=["confidence","center_x", "center_y", "center_z","width","length","height", "yaw"]
    label[numerical_cols] = np.float32(label[numerical_cols].values)
    return label

## main codes

In [8]:
# idx for pair of new_classname and new_cond
mask = (df_logfile['class'] == new_classname) & (df_logfile['cond']== new_cond)
idx = mask.index[mask == True].tolist()[0]

In [9]:
# get the corresponding prediction_folder
prediction_folder =  df_logfile['predictedLabelFolder'][idx]
print("prediction_folder = ")
print(prediction_folder)


# get the old and new submission file name
if idx >= 1:
    old_submissionfile = df_logfile['submissionFile'][idx - 1]
else:
    old_submissionfile = "sample_submission.csv"
new_submissionfile = df_logfile['submissionFile'][idx]

print("\nold: " + old_submissionfile)
print("new: " + new_submissionfile)

prediction_folder = 
../predicted/mirror/epoch_2/pred_boxes3d/Car

old: ../submission/sample_submission_car3bus2ped2.csv
new: ../submission/sample_submission_car4bus2ped2.csv


In [ ]:
df_test=pd.read_csv(old_submissionfile) 
for lyft_id, sample_token in enumerate(df_test.Id):
    
    # predicted label file
    predictedlabel_file = os.path.join(prediction_folder,"{:06n}".format(lyft_id)+'.txt')
    
    if os.path.exists(predictedlabel_file) is not True:
        continue;
    
    print(predictedlabel_file)
    # get the predicted labels 
    predicted_labels = read_label_predict(predictedlabel_file)
    
    # get the existing predicted label of lyft_id
    writelabel_str = df_test.PredictionString[lyft_id]
    # PredictionString[lyft_id] is empty
    if(isinstance(writelabel_str, str) is not True):
        if(math.isnan(writelabel_str)):
            writelabel_str = ""
        else:
            assert False, "PredictionString in old submission file is not str or nan"
    
    # add the new prediction label one by one
    for i in range(len(predicted_labels)):

        args = [predicted_labels.center_x[i],predicted_labels.center_y[i],predicted_labels.center_z[i],
                predicted_labels.width[i],predicted_labels.length[i],predicted_labels.height[i],
                predicted_labels.yaw[i]]
        
        # transform args to lyft
        args =label_global_from_rect(sample_token,args)
        
        
        
        # add the new predicted label: confidence
        writelabel_str = writelabel_str + " " +  str(predicted_labels.confidence[i])
        
        
        # add the new predicted label: x, y, z, w, l, h, yaw
        for argi in range(len(args)):
            writelabel_str = writelabel_str + " " + str(args[argi])

        # # add the new predicted label: class name
        writelabel_str = writelabel_str + " " +  new_classname
            
    # assign the combined writelabel_str to lyft_id
    df_test.PredictionString[lyft_id] = writelabel_str
    
# write to sample_submission.csv
df_test.to_csv(new_submissionfile,index=False)

../predicted/mirror/epoch_2/pred_boxes3d/Car/007370.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007371.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007372.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007373.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007374.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007375.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007376.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007377.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007378.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007379.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007380.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007381.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007382.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007383.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007384.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007385.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007386.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007

/home/lingling/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:4: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



../predicted/mirror/epoch_2/pred_boxes3d/Car/007401.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007402.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007403.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007404.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007405.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007406.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007407.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007408.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007409.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007410.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007411.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007412.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007413.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007414.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007415.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007416.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007417.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007

../predicted/mirror/epoch_2/pred_boxes3d/Car/007571.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007572.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007573.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007574.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007575.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007576.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007577.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007578.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007579.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007580.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007581.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007582.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007583.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007584.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007585.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007586.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007587.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007

../predicted/mirror/epoch_2/pred_boxes3d/Car/007737.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007738.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007739.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007740.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007741.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007742.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007743.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007744.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007745.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007746.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007747.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007748.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007749.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007750.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007751.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007752.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007753.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/007

...

../predicted/mirror/epoch_2/pred_boxes3d/Car/012042.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012043.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012044.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012045.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012046.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012047.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012048.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012049.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012050.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012051.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012052.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012053.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012054.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012055.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012056.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012057.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012058.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012

../predicted/mirror/epoch_2/pred_boxes3d/Car/012214.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012215.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012216.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012217.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012218.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012219.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012220.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012221.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012222.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012223.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012224.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012225.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012226.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012227.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012228.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012229.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012230.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012

../predicted/mirror/epoch_2/pred_boxes3d/Car/012386.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012387.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012388.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012389.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012390.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012391.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012392.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012393.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012394.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012395.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012396.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012397.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012398.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012399.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012400.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012401.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012402.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012

../predicted/mirror/epoch_2/pred_boxes3d/Car/012561.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012562.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012563.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012564.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012565.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012566.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012567.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012568.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012569.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012570.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012571.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012572.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012573.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012574.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012575.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012576.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012577.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012

../predicted/mirror/epoch_2/pred_boxes3d/Car/012734.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012735.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012736.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012737.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012738.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012739.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012740.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012741.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012742.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012743.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012744.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012745.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012746.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012747.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012748.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012749.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012750.txt
../predicted/mirror/epoch_2/pred_boxes3d/Car/012

In [ ]:
df_old = pd.read_csv(old_submissionfile)
df_new = pd.read_csv(new_submissionfile)

In [ ]:
print(prediction_folder)
print(df_old.PredictionString[1624])
print('\n')
print(df_new.PredictionString[1624])